
#### Practica
En base a la practica del COVID vacunación, realizar el proceso de interrupción en base a que el 5% de los pacientes que estan en la sala de espera presentan complicaciones de salud y deben ser trasladados inmediamente al centro de salud más cercano y ser atendidos. Realizar este proceso y generar una segunda linea de simulación para el traslado, atencion y alta del paciente que presenta complicaciones, en base a los siguientes datos:

* Traslado de la ambulancia: 5 - 15 minutos.
* Atención centro de salud: 5 - 24 horas
* Alta: 1 - 2 horas
* El 10% de las personas que presentan complicaciones fallecen.


In [1]:
import simpy
import random

In [3]:
#TRASLADO = random.randint(5,15)
ALTA = random.randint(1,2)
N_PACIENTES = 500
PORCOMPLI = int(N_PACIENTES*0.05)
PER_FALL = int(PORCOMPLI*0.10)
T_COMPLICACION = 300


In [4]:
def tiempo_traslado():
    return random.randint(5,15)

def tiempo_atencion():
    return random.randint(5,24)

def tiempo_alta():
    return random.randint(1,2)
def tiempo_complicacion():
    return random.expovariante(1/T_COMPLICACION)

In [7]:
# Maximo de vehiculos que puede recibir el negocio
MAX_VACUNAS = 57
# Total de numero de mesas para vacunacion
NUM_MESAS = 3
PACIENTES = 300
# Intervalo de tiempo en que llegan vehiculos (minutos)
INTERVALO_LLEGADA = 20 #minutos
# Tiempo de simulación
TIEMPO_SIMULACION = 480 #Minutos = 8 horas
INTERVALO = 10

# Creamos un diccionario para almacenar las horas en que se vacuna los pacientes
tiempo_vacunacion={}
#Diccionario para el numero de pacientes que han sido atendidos
p_vacunado =[]
#Diccionario para el numero de pacientes que no han sido atendidos
p_no_vacunado = []

class Punto(object):
    
    def __init__(self, environment, num_mesas, nombre):
        # Guardamos como variable el entorno de ejecucion
        self.env=environment
        # Creamos el recurso que representa las maquinas
        self.mesas = simpy.Resource(environment, num_mesas)
        self.nombre = nombre
        self.complicacion = False
        
    #Proceso de control, para toma de signos vitales y valida estar dentro del padron
    def control_paciente(self, paciente):
        t_con = random.randint(2,5)
        yield self.env.timeout(t_con)
            
    #Para traslado a la cama/silla de vacunacion y se inyecta
    def vacunar_paciente(self, paciente):
        t_vacunacion = random.randint(5,10)
        print("Tiempo tomado: ",t_vacunacion," para paciente: ",paciente)
        yield self.env.timeout(t_vacunacion)
        
        k = int(paciente.strip('Paciente-'))
        print("Paciente ",k)
        if k in tiempo_vacunacion:
            tiempo_vacunacion[k]=tiempo_vacunacion[k]+1
        else:
            tiempo_vacunacion[k]=t_vacunacion
            
    #Traslado para la salida del paciente
    def salida_paciente(self,paciente):
        t_salida = random.randint(1,3)
        yield self.env.timeout(t_salida)
        
    def complicacion(self):
        while True:
            yield self.env.timeout(tiempo_complicacion())
            if not self.complicacion:
                self.
        
        
       
    
def llegada_paciente(env, nombre,punto):
    # Usamos el reloj de la simulacion (env.now()) para indicar a la
    # hora que llega el vehiculo con el nombre pasado como parametro
    print('Llega el paciente: %s a la hora %.2f.' % (nombre, env.now))
    
    # Especificamos que vamos a usar un recurso (Resource) que representa
    # la maquina de lavado
    with punto.mesas.request() as mesa:
        # Ocupamos la mesa de vacunacion
        yield mesa
        #Indicamos que el paciente se esta trasladando para ser vacunado
        print('Se traslada el paciente: %s a la hora %.2f.' % (nombre,env.now))
        #Procesamos el traslado del paciente
        yield env.process(punto.control_paciente(nombre))
        
        estado = random.randint(1,2)
        print("Sintomas: ",estado)
        if estado == 1:            
            p_no_vacunado.append(nombre)
            
        else:
            # Indicamos que paciente ingresa a la mesa de vacunacion
            print('Ingresa el paciente a las %s a la hora %.2f.' % (nombre, env.now))
            
            # Se realiza la vacunacion
            yield env.process(punto.vacunar_paciente(nombre))
            # Una vez que termina la llamada con 'yield', se indica que se ha lavado el vehiculo
            print('Paciente [%s] vacunado a las %.2f.' % (nombre, env.now))
            p_vacunado.append(nombre)
            
        yield env.process(punto.salida_paciente(nombre))
        print('Paciente [%s] trasladado a la salida a las %.2f.' %(nombre,env.now))
        
        
def ejecutar_simulacion(env,num_mesas, pacientes,intervalo):
    punto = Punto(env,num_mesas,"Cuenca 1")
    for i in range(pacientes):
        env.process(llegada_paciente(env,'Paciente-%d'%(i+1),punto))
    while True:
        yield env.timeout(random.randint(intervalo-10,intervalo+10))
        i+=1
        env.process(llegada_paciente(env,'Paciente-%d'%(i+1),punto))
    
        

In [ ]:
print("Punto de vacunacion Cuenca 1")

# Creamos el entorno de simulacion
env=simpy.Environment()
print("pacientes ", PACIENTES)
env.process(ejecutar_simulacion(env, NUM_MESAS,PACIENTES,INTERVALO))
# Ejecutamos el proceso durante el tiempo de simulacion
env.run(until = TIEMPO_SIMULACION)
